In [143]:
import numpy as np
import ast
ast.literal_eval
import pandas as pd

In [144]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')


In [145]:
movies = movies.merge(credits,on='title')

In [146]:
movies.shape

(4809, 23)

In [147]:
movies = movies[['movie_id' , 'title' , 'overview', 'genres', 'keywords', 'cast','crew']]

In [148]:
movies.dropna(inplace=True)

In [149]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [150]:
movies['keywords'] = movies['keywords'].apply(convert)


In [151]:
def convert3(obj):
    L = []
    counter= 0 
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [152]:
movies['cast'] = movies['cast'].apply(convert3)

In [153]:
movies['genres'] = movies['genres'].apply(convert)

In [154]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i ['job']== 'Director':
            L.append(i['name'])
            break
    return L

In [155]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [156]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [157]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [158]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [159]:
movies['tags']=movies['genres'] + movies['overview'] + movies['keywords'] +movies['cast'] +movies['crew']

In [160]:
new_df = movies[['movie_id','title','tags']]

In [161]:
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

C:\Users\chauh\AppData\Local\Temp\ipykernel_28476\3530946481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))


In [162]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

C:\Users\chauh\AppData\Local\Temp\ipykernel_28476\3679624391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())


In [163]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4806 entries, 0 to 4808
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4806 non-null   int64 
 1   title     4806 non-null   object
 2   tags      4806 non-null   object
dtypes: int64(1), object(2)
memory usage: 150.2+ KB


In [164]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 5000,stop_words='english')

In [165]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [166]:
vectors


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [167]:
import nltk

In [168]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [169]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
    

In [170]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\chauh\AppData\Local\Temp\ipykernel_28476\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [172]:
from sklearn.metrics.pairwise import cosine_similarity

In [174]:
similarity = cosine_similarity(vectors)

In [184]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity [movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [185]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem


In [186]:
import pickle

In [188]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [189]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [190]:
pickle.dump(similarity,open('similarity.pkl','wb'))